In [0]:
%sql
---final code--------------------------------
CREATE OR REPLACE TABLE target_model AS
(
  SELECT
    h.id as source_id,
    h.insurer_id as subscriber_id,
    d.first_name,
    d.middle_name,
    d.last_name,
    case 
      when gender = 'M' then 'Mr.'
      when gender = 'F' and d.marital_status in ('Single','Divorced') then 'Ms.'
      when gender = 'F' and d.marital_status in ('Married','Widowed') then 'Mrs'
      when gender = 'F' and d.marital_status is null then 'Ms.'
      else null
      end as prefix_name,
    case when d.first_name = d.middle_name and h.relationship = 'child' then 'Jr'
          WHEN job_role LIKE '%Engineer%' THEN 'Er'
          WHEN job_role LIKE '%Analyst%' THEN 'Analyst'
          WHEN job_role LIKE '%Manager%' THEN 'Mgr'
          WHEN job_role LIKE '%Administrator%' THEN 'Admin'
          WHEN job_role LIKE '%Developer%' THEN 'Dev'
          WHEN job_role LIKE '%Assistant%' THEN 'Asst'
          WHEN job_role LIKE '%Technician%' THEN 'Tech'
          WHEN job_role LIKE '%Account%' THEN 'Acct'
          WHEN job_role LIKE '%Biostatistician%' THEN 'BioStat'
          WHEN job_role LIKE '%Health Coach%' THEN 'HlthCoach'
          WHEN job_role LIKE '%Designer%' THEN 'Designer'
          WHEN job_role LIKE '%Statistician%' THEN 'Stat'
          WHEN job_role LIKE '%Programmer%' THEN 'Prog'
          WHEN job_role LIKE '%Coordinator%' THEN 'Coord'
          WHEN job_role LIKE '%Automation Specialist%' THEN 'AutoSpec'
          WHEN job_role LIKE '%VP%' THEN 'VP'
          WHEN job_role LIKE '%Geologist%' THEN 'Geol'
          ELSE '(Other)'
        end as suffix_name,
    (COALESCE(prefix_name,'') || ' ' ||d.first_name || ' ' || COALESCE(d.middle_name, '') || ' ' || d.last_name ||' '|| coalesce(suffix_name,'')) AS name,
    'nova_health' as record_source,
    current_timestamp() AS record_created_ts,
    False as is_verified,
    ARRAY_AGG(STRUCT(
      a.address_type,
      a.address_line_1,
      a.address_line_2,
      a.city,
      a.state,
      (case when len(zipcode) > 5 then left(zipcode,5)
            when len(zipcode) = 5 then zipcode end) as zip,
      (case when len(zipcode) > 5 then right(zipcode,4)
            when len(zipcode) = 4 then zipcode end) as post,
      'United States' as country )) AS addresses,
    ARRAY_agg(distinct STRUCT( c.usage_type, c.phone)) AS phones,
    case when d.email like '_%@%_.__%' then d.email else null end as email,
    false as privacy_preference,
    case 
          when LEN(d.ssn) = 11 and d.ssn like '___-__-____' then d.ssn
          when LEN(d.ssn)=9 then substr(d.ssn,1,3)||'-'||substr(d.ssn,4,2)||'-'||substr(d.ssn,6,4) 
          else null end as national_id,
    d.gender,
    d.marital_status,
    d.date_of_birth,
    YEAR(d.date_of_birth) AS year_of_birth,
    CASE WHEN d.deceased_date IS NULL THEN false ELSE true END AS deceased_ind,
    YEAR(to_date(d.deceased_date,'M/d/yyyy')) - year(d.date_of_birth) AS deceased_age,
    to_date(d.deceased_date,'M/d/yyyy') as deceased_date,
     CASE
      WHEN spoken_language_1 IS NULL THEN array(spoken_language_2)
      WHEN spoken_language_2 IS NULL THEN array(spoken_language_1)
      WHEN spoken_language_1 IS NULL AND spoken_language_2 IS NULL THEN NULL
      WHEN spoken_language_2 = spoken_language_1 THEN array(spoken_language_1)
      ELSE ARRAY(spoken_language_1, spoken_language_2)
    END AS languages,
    array(struct(company as employer_name,
                  job_role as employee_role,
                  case when d.deceased_date is not null then 'Inactive' else 'Active' end as employee_status,
                  case when job_hiredate > d.date_of_birth then job_hiredate else null end as employee_hiredate)) as employment,
    map('relationship :',h.relationship,'religion :',d.religion) as additional_source_value

    
  FROM
    header4_json h
  LEFT JOIN
    detail4_csv d ON h.id = d.id
  LEFT JOIN
    contactinfo4_txt c ON h.id = c.id
  LEFT JOIN
    address4_xlsx a ON h.id = a.id
  GROUP BY
    h.id,
    h.insurer_id,
    d.first_name,
    d.middle_name,
    d.last_name,
    d.email,
    d.gender,
    d.marital_status,
    d.date_of_birth,
    d.deceased_date,
    d.spoken_language_1,
    d.spoken_language_2,
    d.company,
    d.job_role,
    d.job_hiredate,
    h.relationship,
    d.religion,
    d.ssn
   
);


num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from target_model 



source_id,subscriber_id,first_name,middle_name,last_name,prefix_name,suffix_name,name,record_source,record_created_ts,is_verified,addresses,phones,email,privacy_preference,national_id,gender,marital_status,date_of_birth,year_of_birth,deceased_ind,deceased_age,deceased_date,languages,employment,additional_source_value
70001,40184,Hettie,null,Keenlayside,Mrs,(Other),Mrs Hettie Keenlayside (Other),nova_health,2024-02-01T11:17:26.950+0000,false,"List(List(Mail, 4307 Ashley Village Suite 758, null, New Kyle, North Dakota, 87337, null, United States), List(Residental, 046 Cox Lights, Suite 600, Griffinhaven, South Carolina, 71709, null, United States))","List(List(Work, (455) 3130004))",jkeenlayside0@disqus.com,false,168-92-1075,F,Widowed,1939-08-05,1939,false,null,null,"List(West Frisian, Swahili)","List(List(Gabcube, Clinical Specialist, Active, 1964-01-29))","Map(relationship : -> child, religion : -> Buddhism)"
70002,40092,Reade,null,Laverenz,Mr.,(Other),Mr. Reade Laverenz (Other),nova_health,2024-02-01T11:17:26.950+0000,false,"List(List(Residental, 737 Banks Row, Apt. 505, North Heather, Arkansas, null, 2341, United States), List(Mail, 183 Dalton Viaduct, Suite 844, South Natalie, Oregon, 77714, null, United States))","List(List(Work, (994) 4561640))",dlaverenz1@senate.gov,false,782-24-9907,M,Widowed,1941-05-14,1941,false,null,null,"List(Swati, Danish)","List(List(Skibox, Staff Scientist, Active, 1958-05-18))","Map(relationship : -> friend, religion : -> Christianity)"
70003,40233,Minnnie,null,Baack,Mrs,(Other),Mrs Minnnie Baack (Other),nova_health,2024-02-01T11:17:26.950+0000,false,"List(List(Mail, 807 Jesus Mills Suite 598, Suite 735, Churchbury, Texas, 97223, null, United States), List(Residental, 27634 Miller Prairie, null, West Ruth, North Dakota, 73151, null, United States))","List(List(Work, (771) 6498755))",dbaack2@sina.com.cn,false,726-01-1271,F,Married,1982-11-20,1982,false,null,null,List(Swati),"List(List(Dabjam, Paralegal, Active, 2011-06-10))","Map(relationship : -> spouse, religion : -> Buddhism)"
70004,40058,Tana,Agata,Aiken,Ms.,VP,Ms. Tana Agata Aiken VP,nova_health,2024-02-01T11:17:26.950+0000,false,"List(List(Residental, 9831 Robert Falls, Apt. 086, Michelleland, Oregon, 05921, 5281, United States), List(Mail, 36717 Philip Common, Suite 278, Thomasborough, Idaho, 91582, 4725, United States))","List(List(Work, (450) 8886723))",aaiken3@nydailynews.com,false,492-62-0968,F,null,1929-02-18,1929,false,null,null,"List(New Zealand Sign Language, Punjabi)","List(List(Aimbu, VP Marketing, Active, 2014-10-08))","Map(relationship : -> spouse, religion : -> null)"
70005,40088,Cyndia,null,Tolomelli,Ms.,Dev,Ms. Cyndia Tolomelli Dev,nova_health,2024-02-01T11:17:26.950+0000,false,"List(List(Mail, 95855 Davis Lodge, Suite 059, Kimberlymouth, Louisiana, 33733, null, United States), List(Residental, 0861 Caldwell Dam, Suite 783, North Robertborough, New Mexico, 31718, null, United States))","List(List(Work, (423) 1700133))",ltolomelli4@istockphoto.com,false,802-24-1062,F,null,1920-05-31,1920,false,null,null,List(Albanian),"List(List(Edgepulse, Senior Developer, Active, 1931-01-16))","Map(relationship : -> friend, religion : -> null)"
70006,40170,Johnny,Renaud,Gibben,Mr.,Asst,Mr. Johnny Renaud Gibben Asst,nova_health,2024-02-01T11:17:26.950+0000,false,"List(List(Residental, 8737 Flores Extension Suite 549, null, Jasonbury, Idaho, 20277, null, United States), List(Mail, 1049 Riggs Stream Suite 632, Suite 465, New Christopher, Maine, 43342, null, United States))","List(List(Work, (334) 1254061))",rgibben5@tumblr.com,false,563-98-1576,M,Single,1958-07-01,1958,false,null,null,List(Georgian),"List(List(Oodoo, Human Resources Assistant I, Active, 2021-12-26))","Map(relationship : -> child, religion : -> Buddhism)"
70007,40194,Judas,null,Mitford,Mr.,Coord,Mr. Judas Mitford Coord,nova_health,2024-02-01T11:17:26.950+0000,false,"List(List(Residental, 7475 Michael Land, Suite 392, New Latoyamouth, Hawaii, 89157, null, United States), List(Ma

In [0]:
%sql
describe target_model;

col_name,data_type,comment
source_id,int,null
subscriber_id,int,null
first_name,string,null
middle_name,string,null
last_name,string,null
prefix_name,string,null
suffix_name,string,null
name,string,null
record_source,string,null
record_created_ts,timestamp,null
